In [10]:
import requests
import json
from pprint import pprint
import pandas as pd
from tabulate import tabulate
import pprint

In [11]:
use_cols = ['Material_ID','Customer','Cal_year_month', 'Gross_Bookings_Qty_CRD']
dtype= {'Material_ID':str,'Customer':str}

In [4]:
bas_s3_pth = 's3://kbs-prediction-service-molex/EUR/'
file_name='monthly_agg_SKU_CUST_EUR.csv'
f'{bas_s3_pth}{file_name}'

's3://kbs-prediction-service-molex/EUR/monthly_agg_SKU_CUST_EUR.csv'

In [4]:
raw_dat = pd.read_csv(f'{bas_s3_pth}{file_name}', 
                      parse_dates=['Cal_year_month'], usecols=use_cols, dtype=dtype)
raw_dat.rename(columns={'Cal_year_month':'Date'}, inplace=True)
raw_dat.tail()

,Material_ID,Customer,Date,Gross_Bookings_Qty_CRD
354290,989974111,159961,2020-09-01,3240.0
354291,989974111,159961,2020-10-01,2200.0
354292,989974111,159961,2020-11-01,1240.0
354293,989974111,159961,2020-12-01,3360.0
354294,989974111,159961,2021-01-01,1680.0


In [5]:
raw_dat['index'] = raw_dat[['Material_ID','Customer']].apply(lambda x: '-'.join(x),axis=1)

In [6]:
proc_dat = raw_dat.groupby(['index', 'Date']).sum().unstack([0]).fillna(0)

In [7]:
proc_dat.columns = proc_dat.columns.droplevel([0])
proc_dat.columns.name =None

In [8]:
mask = proc_dat.apply(lambda x: sum(x!=0)>5, axis=0)

In [9]:
select_cols = proc_dat.columns[mask]

In [10]:
proc_dat_fltr = proc_dat.loc[:,select_cols]

In [11]:
proc_dat_fltr.shape, proc_dat.shape

((97, 9033), (97, 17531))

In [53]:
# proc_dat_fltr.tail()

In [12]:
sav_pth = f'{bas_s3_pth}proc_SKU_CUST_EUR_fltr.csv'
##proc_dat_fltr.to_csv(sav_pth)
sav_pth

's3://kbs-prediction-service-molex/EUR/proc_SKU_CUST_EUR_fltr.csv'

# Setup

In [13]:
Baseurl = 'https://kgsa-dev.kochcloud.com'
Baseurl2 = 'https://kgsa-dev.kochcloud.com/badrul'
user = 'bibin'
password = "3$7AdE9OIJ6M1X*QUfuhD"

In [14]:
## TODO: Read in the data and show the head
fhrDf = pd.read_csv(sav_pth, sep=',', header = 0, index_col = 0)
fhrDf.index = pd.to_datetime(fhrDf.index)
fhrDf.head()

,1041330227-164620,1041340221-164620,1110141230-168170,1480120002-126227,1480120003-126227,1480130002-126227,1480131000-126227,1480170003-126227,1480170004-126227,1480210002-170285,...,989952029-130203,989952029-147125,989952029-155064,989952029-162126,989971002-101011,989971002-101270,989971002-102223,989971002-102440,989971002-103679,989974111-159961
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-02-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,480.0,0.0
2013-04-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-05-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Training Parameters
Dictionary below contains the minimum configuration needed to run a training job.

In [7]:
trainingDict = {
    "master": {
    "exec_environment": "Sequential",
    "data_storage_type": "s3",
    "data_file_type": "csv",
    },
    
    "training": {
    "train_name": "molex-EUROPE-subset-trial",
    "train_description": "Molex EUROPE data training for ARIMA, HOLT, and PROPHET",
    "train_task_type": "Model",
    "train_job_type": "Single-Target",
    "target_data_location": f"{sav_pth}",
    "feature_data_location": "",
    "score_data_location": f"{bas_s3_pth}results/",
    "model_version": "pred-manager-molex-test",
    "data_version": "v1.0",
    "loss_function": "MAPE",
    "train_data_end_dtm": "2020-04-01",
    "test_data_end_dtm": "2020-07-01",
    "validation_data_end_dtm": "2021-01-01",
    "model_location": f"{bas_s3_pth}model/",
    "time_interval": "M",
    "model_names": ["ARIMA", "HOLTWINTERS", "PROPHET"],
    }
}

trainingDict

{'master': {'exec_environment': 'Sequential',
  'data_storage_type': 's3',
  'data_file_type': 'csv'},
 'training': {'train_name': 'molex-EUROPE-subset-trial',
  'train_description': 'Molex EUROPE data training for ARIMA, HOLT, and PROPHET',
  'train_task_type': 'Model',
  'train_job_type': 'Single-Target',
  'target_data_location': 's3://kbs-prediction-service-molex/EUR/proc_SKU_CUST_EUR_fltr.csv',
  'feature_data_location': '',
  'score_data_location': 's3://kbs-prediction-service-molex/EUR/results/',
  'model_version': 'pred-manager-molex-test',
  'data_version': 'v1.0',
  'loss_function': 'MAPE',
  'train_data_end_dtm': '2020-04-01',
  'test_data_end_dtm': '2020-07-01',
  'validation_data_end_dtm': '2021-01-01',
  'model_location': 's3://kbs-prediction-service-molex/EUR/model/',
  'time_interval': 'M',
  'model_names': ['ARIMA', 'HOLTWINTERS', 'PROPHET']}}

### Executing training job

In [8]:
response = requests.post(Baseurl2+"/training", 
                         auth=(user, password),
#                          data = json.dumps(trainingDict))  
                         json = trainingDict)

In [9]:
print(response)
result = response.json()
print(pprint.pprint(result))
trainingId = result['runId']
print(trainingId)

<Response [200]>
{'runId': '36c65a6c-3626-410b-88e1-4c1b8a704e3d', 'status': 'Submitted'}
None
36c65a6c-3626-410b-88e1-4c1b8a704e3d


### Result of Training

In [19]:
status_response = requests.get(Baseurl+"/trainings/" + trainingId,
                              auth=(user,password))    
print(status_response)

<Response [200]>


In [20]:
trainingStatus = status_response.json()
print(pprint.pprint(trainingStatus))

{'runId': '6aa1e5f1-e41f-4d10-9076-ce24352b120e',
 'status': 'Running',
 'trainStartTs': '2021-02-22T19:12:21.326114',
 'updateTs': '2021-02-22T19:11:30.814429'}
None


# Tuning Parameters
Dictionary below contains the minimum configuration needed to run a tuning job.

In [22]:
tuningDict = {
    "master": {
    "exec_environment": "Distributed",
    "data_storage_type": "s3",
    "data_file_type": "csv",
    },
    
    "training": {
        "train_task_type": "Tuning",
        "train_name": "modlex-EUROPE-TUNING",
        "train_description": "Molex APS data training for ARIMA, HOLT, and PROPHET",    
        "train_job_type": "Single-Target",
        "target_data_location": f"{sav_pth}",
        "feature_data_location": "",
        "score_data_location": f"{bas_s3_pth}results",
        "model_version": "pred-manager-molex-test",
        "data_version": "v1.0",
        "loss_function": "MAPE",
        "train_data_end_dtm": "2020-04-01",
        "test_data_end_dtm": "2020-07-01",
        "validation_data_end_dtm": "2021-01-01",
        "model_location": f"{bas_s3_pth}model/",
        "time_interval": "M",
        "model_names": ["ARIMA", "HOLTWINTERS", "PROPHET"],
    },
    "models": {
        "ARIMA": {
        "model_name": "ARIMA",
        "model_time_interval": "M",
        "hyperparam_alg": "GRID-SEARCH",
        "model_config": {
            "parameters": {"p": [0,1,2,3,4,5,6], "d": [0,1,2,3,4,5,6], "q": [0,1,2,3,4,5,6]},
            "hyperparameters": {"disp": 0}}
        },
        'HOLTWINTERS': {
            "model_name": "HOLTWINTERS",
            "model_config": {
                'parameters':{
                    'trend': ['add','mul'],
                    'damped': [True, False],
                    'seasonal_periods':[2,3,4,6,12],
                    'seasonal':[None]
                },
                "hyperparameters": {
                    'smoothing_level':[0.1, 0.2, 0.4, 0.8]
                }
            }
        },
        'PROPHET': {
            "model_name": "PROPHET",
            "model_config":{
                'parameters':{
                    'growth':['linear'],
                    'changepoints': [None],
                    'n_changepoints': [20, 21, 22],
                    'changepoint_range': [0.8, 0.9],
                    'changepoint_prior_scale': [0.05, 0.1],
                    'yearly_seasonality': ['auto'],
                    'weekly_seasonality': ['auto'],
                    'daily_seasonality': ['auto'],
                    'holidays': [None],
                    'seasonality_mode': ['additive','multiplicative'],
                    'seasonality_prior_scale': [10.0],
                    'holidays_prior_scale': [10.0],
                    'interval_width': [0.8],
                    'uncertainty_samples': [200]
                }
            }
        }
    }
}

tuningDict

{'master': {'exec_environment': 'Distributed',
  'data_storage_type': 's3',
  'data_file_type': 'csv'},
 'training': {'train_task_type': 'Tuning',
  'train_name': 'modlex-EUROPE-TUNING',
  'train_description': 'Molex APS data training for ARIMA, HOLT, and PROPHET',
  'train_job_type': 'Single-Target',
  'target_data_location': 's3://kbs-prediction-service-molex/EUR/proc_SKU_CUST_EUR_fltr.csv',
  'feature_data_location': '',
  'score_data_location': 's3://kbs-prediction-service-molex/EUR/results',
  'model_version': 'pred-manager-molex-test',
  'data_version': 'v1.0',
  'loss_function': 'MAPE',
  'train_data_end_dtm': '2020-04-01',
  'test_data_end_dtm': '2020-07-01',
  'validation_data_end_dtm': '2021-01-01',
  'model_location': 's3://kbs-prediction-service-molex/EUR/model/',
  'time_interval': 'M',
  'model_names': ['ARIMA', 'HOLTWINTERS', 'PROPHET']},
 'models': {'ARIMA': {'model_name': 'ARIMA',
   'model_time_interval': 'M',
   'hyperparam_alg': 'GRID-SEARCH',
   'model_config': {'p

In [19]:
Baseurl2 = 'https://kgsa-dev.kochcloud.com/badrul' ##/badrul'
user = 'bibin'
password = "3$7AdE9OIJ6M1X*QUfuhD"

In [20]:
response = requests.post(Baseurl+"/training", 
                         auth=(user, password),
                         data = json.dumps(tuningDict))  
                         #json = trainingDict)

In [21]:
print(response)
##result = response.json()
##print(pprint.pprint(result))
##trainingId = result['runId']
##print(trainingId)
print(response.text)

<Response [200]>
{"runId":"3d8cfe67-e194-4528-9e18-2f17949270a7","status":"Submitted"}


In [54]:
resultResponse = requests.get(Baseurl2+"/trainings/"+ trainingId,
                              auth=(user,password))    
print(resultResponse)
trainResults = resultResponse.json()
print(pprint.pprint(trainResults))

<Response [200]>
{'resultLocation': 's3://kbs-prediction-service-molex/EUR/results/',
 'runId': '2d871cce-70c3-4da8-8a4e-41b5bf1c7782',
 'status': 'Submitted',
 'trainEndTs': None,
 'trainStartTs': '2021-02-22T19:55:39.015731',
 'updateTs': '2021-02-22T19:55:39.007486'}
None


# Prediction or Scoring

### Scoring Parameters
Dictionary below contains the minimum configuration needed to run a scoring job.

In [57]:
f'{bas_s3_pth}{file_name}', Baseurl

('s3://kbs-prediction-service-molex/EUR/monthly_agg_SKU_CUST_EUR.csv',
 'https://kgsa-dev.kochcloud.com')

# Prediction: Using Previously Trained Model

In [28]:
scoringDict = {
        "scoring": {
            "score_name": "Generate forecast for EUR region of molex",
            "target_data_location": sav_pth,
            "model_names": ["ARIMA", "HOLTWINTERS","PROPHET"],
            "prediction_steps": 25,
            "prediction_count": 25,
            "train_run_id": trainingId
        }
    }
predResponse = requests.post(Baseurl+ "/predictions", 
                         auth=(user, password),
                         data = json.dumps(scoringDict))  

In [29]:
print(predResponse)
predStatus = predResponse.json()
print(pprint.pprint(predStatus))
predId = predStatus['runId']
print(predId)

<Response [200]>
{'runId': '20fcd3ab-9f30-4e52-9340-8b48981f1956', 'status': 'Created'}
None
20fcd3ab-9f30-4e52-9340-8b48981f1956


### Get Prediction Status

In [62]:
predResponse = requests.get(Baseurl+"/fetch_predictions/"+ predId,
                              auth=(user,password))    
print(predResponse)
predStatus = predResponse.json()
print(pprint.pprint(predStatus))
predId = predStatus['runId']
print(predId)

NameError: name 'predId' is not defined

### Fetch Prediction Results

In [32]:
predResponse = requests.get(Baseurl+"/predictions/"+ predId,
                              auth=(user,password))    
print(predResponse)
predResults = predResponse.json()
print(pprint.pprint(predResults))

<Response [200]>
{'resultLocation': 's3://prediction-services/score/',
 'runId': '20fcd3ab-9f30-4e52-9340-8b48981f1956',
 'scoreEndTs': '2021-02-17T03:57:54.286754',
 'scoreStartTs': '2021-02-17T03:57:54.109345',
 'status': 'Failed',
 'status_message': 'Failed to write to S3',
 'updateTs': '2021-02-17T03:57:54.028922'}
None
